<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/turkish_predictions_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
pip install sentencepiece 

     |████████████████████████████████| 1.2MB 6.7MB/s 


In [3]:
pip install transformers

     |████████████████████████████████| 1.8MB 6.7MB/s 
     |████████████████████████████████| 890kB 17.5MB/s 
     |████████████████████████████████| 2.9MB 44.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2a7d4a54cf958b412fadf939163f352f17a55300fd9b104b021f1f9c5d5c86c0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import gc
import os
import time
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import date
from transformers import *
from sklearn.metrics import *
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
from torch import Tensor
from torch.optim import *
from torch.nn.modules.loss import *
from torch.optim.lr_scheduler import * 
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler
import pandas as pd



def regular_encode(texts, tokenizer, maxlen=192):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

class Transformer(nn.Module):
    def __init__(self, transformer, num_classes=1):
        """
        Constructor
        
        Arguments:
            model {string} -- Transformer to build the model on. Expects "camembert-base".
            num_classes {int} -- Number of classes (default: {1})
        """
        super().__init__()
        
        self.transformer = transformer

        self.nb_features = self.transformer.pooler.dense.out_features
        # for param in self.transformer.parameters():
        #   param.requires_grad = False
        self.pooler = nn.Sequential(
            nn.Linear(self.nb_features, num_classes), 
            nn.Sigmoid(),
        )


    def forward(self, tokens):
        """
        Usual torch forward function
        
        Arguments:
            tokens {torch tensor} -- Sentence tokens
        
        Returns:
            torch tensor -- Class logits
        """
        hidden_states = self.transformer(
            tokens, attention_mask=(tokens > 0).long()
        )[1]

        # hidden_states = hidden_states[-1][:, 0] # Use the representation of the first token of the last layer

        ft = self.pooler(hidden_states)

        return ft


In [6]:
class bce(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(bce, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        one=(1-targets)*torch.log(1-inputs)
        zero=(targets*torch.log(inputs))
        loss = torch.mean((one+zero)*-1)
        
        return loss
class JigsawDataset:
    """
    Torch dataset for training and validating
    """
    def __init__(self, x,y,is_test):
        super().__init__()
        self.y = y 
        self.is_test=is_test
        self.sentences = x
        

    def __len__(self):
        return self.sentences.shape[0]

    def __getitem__(self, idx):
      len=self.__len__()
      if idx>len:
        idx=idx%len
      if self.is_test==0:
        return torch.tensor(self.sentences[idx]), torch.tensor(self.y[idx]).float()
      else:
        return torch.tensor(self.sentences[idx])


In [7]:
from sklearn.model_selection import train_test_split
from statistics import mean
import torch_xla
from sklearn.preprocessing import *
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
import gc
import os
import cv2
import zipfile
from pathlib import Path
import random
import argparse
import sys
from statistics import mean
import yaml
from tqdm.notebook import tqdm
import time
import albumentations as A
from sklearn.model_selection import StratifiedKFold
import random
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from typing import Dict
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt

def train_all(train_loader, model, device, optimizer):
    model.train()
    # t = tqdm(train_loader, disable=not xm.is_master_ordinal())
    model.train()
    lss=bce()
    loss1=[]
    for step, (x, y_batch) in enumerate(train_loader): 
            
            # x = x.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x)
            
            loss = lss(y_pred.view(-1).float(), y_batch.float())
            loss.backward()
            loss1.append(loss.item())
            xm.optimizer_step(optimizer)
            
            model.zero_grad()
    return mean(loss1)

def valid_all(train_loader, model, device):
    # t = tqdm(train_loader, disable=not xm.is_master_ordinal())
    lss=bce()
    loss1=[]
    for step, (x, y_batch) in enumerate(train_loader): 
            
            # x = x.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x)
            
            loss = lss(y_pred.view(-1).float(), y_batch.float())
            loss1.append(loss.item())
            
    return mean(loss1)

def predict_all(train_loader, model,device,df,batch_size):
    # t = tqdm(train_loader, disable=not xm.is_master_ordinal())
    predict=[]
    for step, (x) in tqdm(enumerate(train_loader),total=df.shape[0]/(batch_size)): 
            
            y_pred = model(x.to(device))
            predict.append(y_pred)
            gc.collect()
    return predict


def load_data(lang):
    tst=pd.read_csv('/content/gdrive/My Drive/multilingual/test.csv.zip',usecols=['lang','content'])  
    return tst.loc[tst['lang']==lang].reset_index(drop=True).drop(['lang'],1)


def get_lang(val,tst,lang):
  df=pd.concat([val,tst],0)
  return df.loc[df['lang']==lang].reset_index(drop=True).drop(['id','lang'],1)

def main():
    epochs=1
    seed=42
    batch_size=16
    # Setting seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

    df=load_data('tr')
    tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
    x_train = regular_encode(list(df.content.values), tokenizer, maxlen=192)
    print(df.shape)



    def run():

        torch.manual_seed(seed)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")
        model=Transformer(model).to(device)
        model.load_state_dict(torch.load( '/content/gdrive/My Drive/turkish')['state_dict'])

        #Training
        train_dataset = JigsawDataset(x_train,None,1)


        train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,
            drop_last=False,
            num_workers=2
        )

        

        
        predictions=[]
        predictions=predict_all(train_loader,model,device,df,batch_size)



        np.save('predictions.npy',predictions)
            


    def _mp_fn(rank, flags):
        torch.set_default_tensor_type('torch.FloatTensor')
        run()
        
    FLAGS={}
    run()

In [ ]:
main()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(14000, 1)


 12%|█▏        | 13/109.375 [00:19<03:06,  1.93s/it]